In [24]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
base_dir = os.getcwd()

## Load data 

In [25]:
data_filename = "m29_7arcmin_panstarrs_data.csv"
data = pd.read_csv(os.path.join(base_dir, data_filename), sep=';')
useful_columns = ["_r", "RAJ2000", "DEJ2000", "gmag", "e_gmag", "rmag", "e_rmag", "imag", "e_imag", "zmag", "e_zmag"]
data = data[useful_columns]

def to_float(x): 
    try: 
        return float(x)
    except Exception:
        return np.nan 
data = data.applymap(to_float).astype(float)

In [26]:
data.dropna(subset=["gmag", "rmag", "imag", "zmag"], inplace=True)
print(data.describe())

                _r      RAJ2000      DEJ2000         gmag       e_gmag  \
count  2641.000000  2641.000000  2641.000000  2641.000000  1629.000000   
mean      4.725604   305.935539    38.483476    21.623468     0.055435   
std       1.592680     0.075223     0.058280     3.276687     0.065481   
min       0.034900   305.794028    38.371720     8.725000     0.000000   
25%       3.592300   305.873864    38.435752    19.892500     0.009100   
50%       4.972200   305.929754    38.482309    21.618000     0.028400   
75%       6.075000   305.994990    38.529979    23.485000     0.070900   
max       6.999800   306.091144    38.602637    30.090000     0.432200   

              rmag       e_rmag         imag       e_imag         zmag  \
count  2641.000000  2131.000000  2641.000000  2451.000000  2641.000000   
mean     19.719195     0.036523    18.562242     0.022226    17.803363   
std       2.478741     0.049236     2.039345     0.034142     1.803760   
min       8.338800     0.000000     8

In [27]:
def ugriz_to_ubvri(ugriz):
    u, g, r, i, z = ugriz
    u = np.array(u)
    g = np.array(g)
    r = np.array(r)
    i = np.array(i)
    z = np.array(z)

    V = g - 0.59*(g - r) - 0.01
    B = g + 0.39*(g - r) + 0.21
    R = V - (1.09*(r - i) + 0.22)
    U = 0.77*(u - g) - 0.88 + B

    I = R - (r - i + 0.21)
    return (U, B, V, R, I)

gmags = data["gmag"]
umags = pd.Series([69 for _ in range(len(gmags))])
rmags = data["rmag"]
imags = data["imag"]
zmags = data["zmag"]
_, data["B"], data["V"], data["R"], data["I"] = ugriz_to_ubvri((umags, gmags, rmags, imags, zmags))

## Apply distance correction, reddening

In [28]:
m29_dist = 1148 # pc
obs_minus_absolue = 5*np.log10(m29_dist) - 5
data = data[data["V"] < 9 + obs_minus_absolue]
data["B"] -= obs_minus_absolue
data["V"] -= obs_minus_absolue
data["R"] -= obs_minus_absolue
data["I"] -= obs_minus_absolue
a_redd = 0.744

A_v = 3.1*a_redd
A_b = 1.32*A_v
A_r = 0.79*A_v
A_i = 0.55*A_v

data["B"] -= A_b
data["V"] -= A_v 
data["R"] -= A_r
data["I"] -= A_i

## Load isochrones

In [33]:
times = [6.0 + i*0.1 for i in range(31)]

filenames = [f"isochrone_{10*t:.0f}.dat" for t in times]
isochrone_cols = ["M_ini", "MV", "U-B", "B-V", "V-R", "V-I"]
isochrones = {
    times[i]: pd.read_csv(os.path.join("..", "isochrones", filenames[i]), delim_whitespace=True)[isochrone_cols] for i in range(len(filenames))
}

low_mass_filename = [f"lm_isochrone_{10*t:.0f}.dat" for t in times]
lm_iso_cols = ["M_ini","BC", "U-B", "B-V", "V-R", "V-I"]
low_mass_isochrones = {
    times[i]: pd.read_csv(os.path.join("..", "isochrones", low_mass_filename[i]), delim_whitespace=True)[lm_iso_cols] for i in range(len(filenames))
}
print(low_mass_isochrones[6].columns)


Index(['M_ini', 'BC', 'U-B', 'B-V', 'V-R', 'V-I'], dtype='object')


In [38]:
%matplotlib qt
from matplotlib import colors

times = [6.0 + 0.1*i for i in range(31)]
final_isochrones = {}
for index, log_t in enumerate(times):
    vmags = isochrones[log_t]["MV"] 
    bmags = isochrones[log_t]["B-V"] + vmags
    rmags = vmags - isochrones[log_t]["V-R"]
    imags = vmags - isochrones[log_t]["V-I"]
    umags = isochrones[log_t]["U-B"] + bmags 
    isochrones[log_t]["isoU"] = umags
    isochrones[log_t]["isoB"] = bmags
    isochrones[log_t]["isoV"] = vmags
    isochrones[log_t]["isoR"] = rmags
    isochrones[log_t]["isoI"] = imags
    
    lmb = low_mass_isochrones[log_t]["BC"]
    lmu = low_mass_isochrones[log_t]["U-B"] + lmb
    lmv = lmb - low_mass_isochrones[log_t]["B-V"]
    lmr = lmv - low_mass_isochrones[log_t]["V-R"]
    lmi = lmv - low_mass_isochrones[log_t]["V-I"]
    low_mass_isochrones[log_t]["isoU"] = lmu 
    low_mass_isochrones[log_t]["isoB"] = lmb 
    low_mass_isochrones[log_t]["isoV"] = lmv 
    low_mass_isochrones[log_t]["isoR"] = lmr 
    low_mass_isochrones[log_t]["isoI"] = lmi 
    
    low_mass_isochrones[log_t] = low_mass_isochrones[log_t][low_mass_isochrones[log_t]["M_ini"] < 0.5]  
    final_isochrones[log_t] = pd.concat((low_mass_isochrones[log_t], isochrones[log_t]))

print(final_isochrones[7.5].describe())

# print(low_mass_isochrones[6].columns)
# print(isochrones[6].columns)

def plot_3_cmds(data):
    fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3)
    B = data["B"]
    V = data["V"]
    R = data["R"]
    I = data["I"]

# ax1 B-V vs V
    ax1.scatter(B-V, V, s=1)
    ax1.set_xlabel("B-V")
    ax1.set_ylabel("V")

# ax2 V-R vs R
    ax2.scatter(V-R, R, s=1)
    ax2.set_xlabel("V-R")
    ax2.set_ylabel("R")

# ax3 R-I vs I
    ax3.scatter(R-I, I, s=1)
    ax3.set_xlabel("R-I")
    ax3.set_ylabel("I")

    plt.tight_layout()
# isochrone
    cdict = {'red':   ((0.0,  0.22, 0.0),
                       (0.5,  1.0, 1.0),
                       (1.0,  0.89, 1.0)),

             'green': ((0.0,  0.49, 0.0),
                       (0.5,  1.0, 1.0),
                       (1.0,  0.12, 1.0)),

             'blue':  ((0.0,  0.72, 0.0),
                       (0.5,  0.0, 0.0),
                       (1.0,  0.11, 1.0))}

    cmap = colors.LinearSegmentedColormap('custom', cdict)

    times = [6.0 + 0.1*i for i in range(31)]
    for index, log_t in enumerate(times):
        # vmags = isochrones[log_t]["MV"] 
        # bmags = isochrones[log_t]["B-V"] + vmags
        # rmags = vmags - isochrones[log_t]["V-R"]
        # imags = vmags - isochrones[log_t]["V-I"]
        # umags = isochrones[log_t]["U-B"] + bmags 
        # isochrones[log_t]["isoU"] = umags
        # isochrones[log_t]["isoB"] = bmags
        # isochrones[log_t]["isoV"] = vmags
        # isochrones[log_t]["isoR"] = rmags
        # isochrones[log_t]["isoI"] = imags
        ax1.plot(bmags - vmags, bmags, c=cmap(index/len(times)))
        ax2.plot(vmags - rmags, vmags, c=cmap(index/len(times)))
        ax3.plot(rmags - imags, rmags, c=cmap(index/len(times)))
    ax1.set_xlim((-0.5, 2.5))
    ax1.set_ylim((-2, 7.5))

    ax2.set_xlim((-0.5, 2))
    ax2.set_ylim((-2, 7))

    ax3.set_ylim((-2, 6.5))
    ax3.set_xlim((-1, 1.5))
    ax1.invert_yaxis()
    ax2.invert_yaxis()
    ax3.invert_yaxis()

           M_ini         BC        U-B        B-V        V-R        V-I  \
count  74.000000  23.000000  74.000000  74.000000  74.000000  74.000000   
mean    1.169162  -1.795157   0.597977   0.826696   0.531407   1.096185   
std     0.920855   0.244455   0.630200   0.652434   0.426995   0.949896   
min     0.200000  -2.174500  -0.413300  -0.132900  -0.045400  -0.157700   
25%     0.417250  -1.997100   0.050500   0.125175   0.083375   0.166350   
50%     0.845500  -1.801300   0.646300   0.914350   0.506150   0.931950   
75%     1.720500  -1.599650   1.229075   1.508000   0.993125   2.100650   
max     3.500000  -1.390700   1.308000   1.601000   1.129700   2.585600   

            isoU       isoB       isoV       isoR       isoI         MV  
count  74.000000  74.000000  74.000000  74.000000  74.000000  51.000000  
mean    3.225914   2.627936   1.801241   1.269834   0.705055   4.121516  
std     4.090401   4.057389   4.177176   4.353825   4.675518   2.787940  
min    -0.997000  -2.174500 

## Metric in UBVRI space 
attempt 1: metric is of data scaled so that its std is 1 (approx the same shape in all dimensions)

In [8]:
def metric1(color1, color2, stds):
    """
    color is the tuple (U,B,V,R,I), or (B, V, R, I)
    stds is a list of the same shape as colors, and contains the stds of the corresponding colors
    """
    return sum([((color1[i] - color2[i])/stds[i])**2 for i in range(len(color1))])**0.5

In [9]:
from matplotlib import colors
%matplotlib qt
fig = plt.figure()
ax = plt.axes(projection="3d")
# catalogue data
# outliers identified by eye
data = data[data["B"] > 0]
data = data[data["V"] > 0]
data = data[data["R"] > 0]
B = data["B"]
V = data["V"]
R = data["R"]
I = data["I"]
ax.scatter3D(B, V, R, s=1, color='red')

# isochrone
cdict = {'red':   ((0.0,  0.22, 0.0),
                   (0.5,  1.0, 1.0),
                   (1.0,  0.89, 1.0)),

         'green': ((0.0,  0.49, 0.0),
                   (0.5,  1.0, 1.0),
                   (1.0,  0.12, 1.0)),

         'blue':  ((0.0,  0.72, 0.0),
                   (0.5,  0.0, 0.0),
                   (1.0,  0.11, 1.0))}

cmap = colors.LinearSegmentedColormap('custom', cdict)

times = [6, 7, 8, 9]
for index, log_t in enumerate(times):
    vmags = isochrones[log_t]["MV"] 
    bmags = isochrones[log_t]["B-V"] + vmags
    rmags = vmags - isochrones[log_t]["V-R"]
    imags = vmags - isochrones[log_t]["V-I"]

    ax.plot3D(bmags, vmags, rmags, c=cmap(index/len(times)))
ax.set_xlabel("B mag")
ax.set_ylabel("V mag")
ax.set_zlabel("R mag")
plt.show()
print(data.columns)

Index(['_r', 'RAJ2000', 'DEJ2000', 'gmag', 'e_gmag', 'rmag', 'e_rmag', 'imag',
       'e_imag', 'zmag', 'e_zmag', 'B', 'V', 'R', 'I'],
      dtype='object')


## Plot n_stars below a certain distance 

In [19]:
stds = [np.std(data[filter]) for filter in ["B", "V", "R", "I"]]
print(stds)
right_isochrone = isochrones[7.1]
print(right_isochrone.columns)

distance_data = []
mass_data = []
print(len(data["B"]))
for i in range(len(data["B"])):
    # index of isochrone where the distance is minimal
    min_index = np.argmin([
        metric1(
            (data["B"].iloc[i], data["V"].iloc[i], data["R"].iloc[i], data["I"].iloc[i]),
            (right_isochrone["isoB"].iloc[j], right_isochrone["isoV"].iloc[j], right_isochrone["isoR"].iloc[j], right_isochrone["isoI"].iloc[j]), 
            stds
        ) for j in range(len(right_isochrone["isoB"]))
    ])
    mass_data.append(right_isochrone["M_ini"][min_index])
    # dist = np.min([
    #     metric1(
    #         (data["B"].iloc[i], data["V"].iloc[i], data["R"].iloc[i], data["I"].iloc[i]),
    #         (right_isochrone["isoB"].iloc[j], right_isochrone["isoV"].iloc[j], right_isochrone["isoR"].iloc[j], right_isochrone["isoI"].iloc[j]), 
    #         stds
    #     ) for j in range(len(right_isochrone["isoB"]))
    # ])
    dist = metric1(
        (data["B"].iloc[i], data["V"].iloc[i], data["R"].iloc[i], data["I"].iloc[i]), 
        (right_isochrone["isoB"].iloc[min_index], right_isochrone["isoV"].iloc[min_index], right_isochrone["isoR"].iloc[min_index], right_isochrone["isoI"].iloc[min_index])
    )
    distance_data.append(dist)

data["distance"] = distance_data
data["mass"] = mass_data

[1.8657112086230354, 1.5600759542416949, 1.4411258827436177, 1.5674809337549491]
Index(['M_ini', 'MV', 'U-B', 'B-V', 'V-R', 'V-I', 'isoU', 'isoB', 'isoV',
       'isoR', 'isoI'],
      dtype='object')
675


TypeError: metric1() missing 1 required positional argument: 'stds'

In [22]:
%matplotlib qt
plt.yscale('log')
plt.xscale('log')
plt.hist(data["mass"], bins=50, range=[0.5, 4])
plt.show()
threshold = 0.25
data_in_cluster = data[data["distance"] < threshold]

print(data.describe())

               _r     RAJ2000     DEJ2000        gmag      e_gmag        rmag  \
count  675.000000  675.000000  675.000000  675.000000  652.000000  675.000000   
mean     4.627891  305.934188   38.483931   18.071929    0.010234   16.848325   
std      1.591300    0.074790    0.056365    1.734367    0.012759    1.465992   
min      0.293800  305.796876   38.371720   13.095000    0.000000   12.468000   
25%      3.409650  305.873374   38.439147   16.962150    0.004200   15.987800   
50%      4.883600  305.929142   38.482580   18.496000    0.007500   17.183000   
75%      6.007650  305.991380   38.526366   19.498300    0.012925   18.046900   
max      6.999800  306.088732   38.599918   20.815300    0.242400   19.894000   

           e_rmag        imag      e_imag        zmag      e_zmag           B  \
count  646.000000  675.000000  643.000000  675.000000  653.000000  675.000000   
mean     0.006957   16.245478    0.007733   15.830681    0.006086    5.414977   
std      0.008833    1.3882